# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options

### Add year of the exam and topic 

In [3]:
year: int = 2012
topic: str = "BIOLOGÍA"
exam_acronym: str = "bir"
total_num_rows: int
num_questions: int
total_num_rows, num_questions= compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_2/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(1, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,- 2 -
1,1. La fase de meseta del potencial de acción c...
2,díaco:
3,
4,1.
5,Es característica de las células del nodo sinu-
6,sal.
7,2.
8,Se origina por activación de canales de K +.
9,3. Se debe a una conductancia al Na + no in...


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_2/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.txt", sep= " ", header=None)
raw_answers_df.head(10)

,0,1,2,3,4,5,6,7,8,9
0,1,2,3,4,5,6,7,8.0,9.0,10
1,4,4,4,1,1,2,4,4.0,5.0,5
2,21,22,23,24,25,26,27,28.0,29.0,30
3,5,3,2,2,5,1,5,1.0,4.0,1
4,41,42,43,44,45,46,47,48.0,49.0,50
5,4,2,1,5,2,1,A,3.0,4.0,3
6,61,62,63,64,65,66,67,68.0,69.0,70
7,3,A,3,4,2,3,1,2.0,3.0,1
8,81,82,83,84,85,86,87,88.0,89.0,90
9,4,1,3,4,1,1,3,1.0,1.0,1


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0    1. La fase de meseta del potencial de acción car-
1                                               díaco:
2                                                   1.
3      Es característica de las células del nodo sinu-
4                                                 sal.
5                                                   2.
6         Se origina por activación de canales de K +.
7    3.  Se debe a una conductancia al Na +  no ina...
8                                               vante.
9    4.  Se origina por activación de canales de Ca...
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [11]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [12]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. La fase de meseta del potencial de acción c...
1    1. Es característica de las células del nodo s...
2      2. Se origina por activación de canales de K +.
3    3.  Se debe a una conductancia al Na +  no ina...
4    4.  Se origina por activación de canales de Ca...
5    5.  Está medida por la inactivación de canales...
6                        2. Una solución 50 mM de CaCl
7                        2 es isoosmótica con otra de:
8                                                   1.
9                                          50 mM NaCl.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [13]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 20
Number of rows not ending with '.', ':' or '?' = 14
[6, 77, 116, 338, 344, 405, 1048, 1143, 1145, 1239, 1241, 1258, 1327, 1329]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [14]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [15]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [22]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [6, 8, 116, 405, 557, 882, 967, 980, 993, 1030, 1043, 1050, 1143, 1145,1239, 1241, 1258, 1271, 1327, 1329]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 6:
4    4.  Se origina por activación de canales de Ca...
5    5.  Está medida por la inactivación de canales...
6                        2. Una solución 50 mM de CaCl
7                        2 es isoosmótica con otra de:
8                                                   1.
Name: text, dtype: object
Id to fix 77:
75               1. Son terminaciones nerviosas libres.
76    2.  Están inervados por fibras mielínicas de g...
77       3. Responden a cambios térmico entre 20 y 40ºC
78         4.  Su proteína receptora es la transducina.
79                          5.  Son receptores fásicos.
Name: text, dtype: object
Id to fix 116:
114                    4.  La libera la mucosa duodenal.
115                  5.  Inhibe el esfínter íleco-cecal.
116    20. La principal hormona implicada en la secre...
117                                             3 -  es:
118                                  1. Colecistocinina.
Name: text, dtype: object
Id to fix 338:
336                      

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

If the number is diferent it raises a Warning

In [23]:
print(f"Correct number of rows = {total_num_rows}\nNumber of rows in df = {exam_df_fixed.shape[0]}")
if total_num_rows != exam_df_fixed.shape[0]:
    raise Warning("More or Less rows than expected")

Correct number of rows = 1410
Number of rows in df = 1410


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [24]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [25]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [26]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, num_questions + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [27]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [28]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,Option_5
0,1. La fase de meseta del potencial de acción c...,1. Es característica de las células del nodo s...,2. Se origina por activación de canales de K +.,3. Se debe a una conductancia al Na + no ina...,4. Se origina por activación de canales de Ca...,5. Está medida por la inactivación de canales...
1,2. Una solución 50 mM de CaCl2 es isoosmótica ...,1.50 mM NaCl.,2. 100 mM urea.,3. 150 mM NaCl.,4. 150 mM urea.,5. 100 mM NaCl.
2,3. Las sinapsis eléctricas:,1. Permiten sólo el paso de iones.,2. Presentan gran retardo sináptico.,3. Son unidireccionales.,4. Están medidas por conexones.,5. Están en la unión neuromuscular.
3,4. Permea Cl - el receptor:,1. Glicinérgico.,2. Nicotínico.,3. Adrenérgico alfa.,4. Glutamatérgico tipo NMDA.,5. Glutamatérgico tipo AMPA.
4,5. La corteza motora primaria:,1. Está en la circunvolución precental.,2. Evoca contracción de músculos ipsilaterales.,3. Sus principales eferencias directas son al...,4. Recibe aferencias exclusivamente del área ...,5. Su lesión no impide realizar movimientos v...
5,6. Es un núcleo motor troncoencefálico:,1. Putamen.,2. Núcleo rojo.,3. Fastigial.,4. Caudado.,5. Dentado.
6,7. En el reflejo tendinoso de Golgi se inhiben:,1. Los órganos tendinosos de Golgi.,2. Las fibras aferentes Ia.,3. Las fibras aferentes Ib.,4. Las motoneuronas α.,5. Las motoneuronas γ.
7,8. Las fibras trepadoras de la corteza cerebel...,1. Son inhibidoras.,2. Proceden de los núcleos pontinos.,3. Se proyectan en las células granulosas.,4. Sinaptan directamente con las células de P...,5. Forman las fibras paralelas.
8,"9. Los canales semicirculares, el utrículo y e...",1. Presentar otolitos.,2. Detectar la aceleración lineal.,3. Detectar la aceleración angular.,4. Tener un extremo llamado ampolla.,5. Estar llenos de endolinfa.
9,10. Son amielínicas las fibras:,1. A alfa.,2. A beta.,3. A gamma.,4. B.,5. C.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Initial Cleaning and Numeric Conversion:**

* Iterates through all columns of the `raw_answers_df` DataFrame.
* For string-type columns, replaces all occurrences of "A" with "0".
* Non-string columns are ignored.
* Applies `pd.to_numeric` to all columns, converting values to numbers (floats) and handling conversion errors with `errors='coerce'`, which replaces non-numeric values with `NaN`.
* Replaces all `NaN` values with `999.9`.
* Displays the first few rows of the transformed DataFrame (`raw_answers_df_2`).

In [29]:
for col in raw_answers_df:
    try:
        raw_answers_df[col] = raw_answers_df[col].str.replace("A", "0")
    except AttributeError:
        pass

raw_answers_df_2 = raw_answers_df.apply(pd.to_numeric, errors= 'coerce')
raw_answers_df_2 = raw_answers_df_2.fillna(999.9)
raw_answers_df_2.head(10)

,0,1,2,3,4,5,6,7,8,9
0,1,2,3,4,5,6.0,7.0,8.0,9.0,10.0
1,4,4,4,1,1,2.0,4.0,4.0,5.0,5.0
2,21,22,23,24,25,26.0,27.0,28.0,29.0,30.0
3,5,3,2,2,5,1.0,5.0,1.0,4.0,1.0
4,41,42,43,44,45,46.0,47.0,48.0,49.0,50.0
5,4,2,1,5,2,1.0,0.0,3.0,4.0,3.0
6,61,62,63,64,65,66.0,67.0,68.0,69.0,70.0
7,3,0,3,4,2,3.0,1.0,2.0,3.0,1.0
8,81,82,83,84,85,86.0,87.0,88.0,89.0,90.0
9,4,1,3,4,1,1.0,3.0,1.0,1.0,1.0


In [30]:
raw_answers_df_2.tail(10)

,0,1,2,3,4,5,6,7,8,9
38,151,152,153,154,155,156.0,157.0,158.0,159.0,160.0
39,3,5,5,3,1,4.0,4.0,2.0,5.0,3.0
40,171,172,173,174,175,176.0,177.0,178.0,179.0,180.0
41,5,2,4,3,4,2.0,4.0,3.0,2.0,1.0
42,191,192,193,194,195,196.0,197.0,198.0,199.0,200.0
43,1,1,2,5,4,2.0,3.0,1.0,5.0,0.0
44,211,212,213,214,215,216.0,217.0,218.0,219.0,220.0
45,4,1,4,5,3,3.0,2.0,2.0,4.0,5.0
46,231,232,233,234,235,999.9,999.9,999.9,999.9,999.9
47,4,3,1,1,4,999.9,999.9,999.9,999.9,999.9


**2. Transformation to "Longer" Format:**

* Creates two empty lists, number_question_answers and number_question_correct.
* Iterates through the rows of raw_answers_df_2 in steps of two.
* For each pair of rows, the values from the first row are appended to the number_question_answers list, and the values from the second row are appended to the number_question_correct list.
* Creates a new DataFrame raw_answers_df_longer with the number_question_answers and number_question_correct lists as columns "V" and "RC", respectively.
* Displays the first few rows of raw_answers_df_longer.

In [31]:
number_question_answers: list = []
number_question_correct: list = []

for i in range(0, len(raw_answers_df_2), 2):
    number_question_answers.extend(raw_answers_df_2.iloc[i].tolist())
    number_question_correct.extend(raw_answers_df_2.iloc[i + 1].tolist())

raw_answers_df_longer: pd.DataFrame = pd.DataFrame({"V": number_question_answers, "RC": number_question_correct})
raw_answers_df_longer.head(10)

,V,RC
0,1.0,4.0
1,2.0,4.0
2,3.0,4.0
3,4.0,1.0
4,5.0,1.0
5,6.0,2.0
6,7.0,4.0
7,8.0,4.0
8,9.0,5.0
9,10.0,5.0


**3. Final Sorting and Cleaning:**

* Sorts the raw_answers_df_longer DataFrame by the "V" column.
* Replaces all 999.9 values with NaN.
* Removes all rows containing NaN.
* Displays the first few rows of the cleaned DataFrame (answers_df_clean).

In [32]:
raw_answers_df_sorted = raw_answers_df_longer.sort_values(by="V")
raw_answers_df_sorted = raw_answers_df_sorted.replace(999.9, np.nan)
answers_df_clean = raw_answers_df_sorted.dropna()
answers_df_clean = answers_df_clean.reset_index(drop=True)
answers_df_clean.head(10)

,V,RC
0,1.0,4.0
1,2.0,4.0
2,3.0,4.0
3,4.0,1.0
4,5.0,1.0
5,6.0,2.0
6,7.0,4.0
7,8.0,4.0
8,9.0,5.0
9,10.0,5.0


In [33]:
answers_df_clean.tail(10)

,V,RC
225,226.0,1.0
226,227.0,1.0
227,228.0,5.0
228,229.0,4.0
229,230.0,3.0
230,231.0,4.0
231,232.0,3.0
232,233.0,1.0
233,234.0,1.0
234,235.0,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [34]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,Option_5,RC
0,1. La fase de meseta del potencial de acción c...,1. Es característica de las células del nodo s...,2. Se origina por activación de canales de K +.,3. Se debe a una conductancia al Na + no ina...,4. Se origina por activación de canales de Ca...,5. Está medida por la inactivación de canales...,4.0
1,2. Una solución 50 mM de CaCl2 es isoosmótica ...,1.50 mM NaCl.,2. 100 mM urea.,3. 150 mM NaCl.,4. 150 mM urea.,5. 100 mM NaCl.,4.0
2,3. Las sinapsis eléctricas:,1. Permiten sólo el paso de iones.,2. Presentan gran retardo sináptico.,3. Son unidireccionales.,4. Están medidas por conexones.,5. Están en la unión neuromuscular.,4.0
3,4. Permea Cl - el receptor:,1. Glicinérgico.,2. Nicotínico.,3. Adrenérgico alfa.,4. Glutamatérgico tipo NMDA.,5. Glutamatérgico tipo AMPA.,1.0
4,5. La corteza motora primaria:,1. Está en la circunvolución precental.,2. Evoca contracción de músculos ipsilaterales.,3. Sus principales eferencias directas son al...,4. Recibe aferencias exclusivamente del área ...,5. Su lesión no impide realizar movimientos v...,1.0
5,6. Es un núcleo motor troncoencefálico:,1. Putamen.,2. Núcleo rojo.,3. Fastigial.,4. Caudado.,5. Dentado.,2.0
6,7. En el reflejo tendinoso de Golgi se inhiben:,1. Los órganos tendinosos de Golgi.,2. Las fibras aferentes Ia.,3. Las fibras aferentes Ib.,4. Las motoneuronas α.,5. Las motoneuronas γ.,4.0
7,8. Las fibras trepadoras de la corteza cerebel...,1. Son inhibidoras.,2. Proceden de los núcleos pontinos.,3. Se proyectan en las células granulosas.,4. Sinaptan directamente con las células de P...,5. Forman las fibras paralelas.,4.0
8,"9. Los canales semicirculares, el utrículo y e...",1. Presentar otolitos.,2. Detectar la aceleración lineal.,3. Detectar la aceleración angular.,4. Tener un extremo llamado ampolla.,5. Estar llenos de endolinfa.,5.0
9,10. Son amielínicas las fibras:,1. A alfa.,2. A beta.,3. A gamma.,4. B.,5. C.,5.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [35]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [36]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
Option_5    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [37]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [38]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,Option_5,RC,year
0,1. La fase de meseta del potencial de acción c...,1. Es característica de las células del nodo s...,2. Se origina por activación de canales de K +.,3. Se debe a una conductancia al Na + no ina...,4. Se origina por activación de canales de Ca...,5. Está medida por la inactivación de canales...,4,2012
1,2. Una solución 50 mM de CaCl2 es isoosmótica ...,1.50 mM NaCl.,2. 100 mM urea.,3. 150 mM NaCl.,4. 150 mM urea.,5. 100 mM NaCl.,4,2012
2,3. Las sinapsis eléctricas:,1. Permiten sólo el paso de iones.,2. Presentan gran retardo sináptico.,3. Son unidireccionales.,4. Están medidas por conexones.,5. Están en la unión neuromuscular.,4,2012
3,4. Permea Cl - el receptor:,1. Glicinérgico.,2. Nicotínico.,3. Adrenérgico alfa.,4. Glutamatérgico tipo NMDA.,5. Glutamatérgico tipo AMPA.,1,2012
4,5. La corteza motora primaria:,1. Está en la circunvolución precental.,2. Evoca contracción de músculos ipsilaterales.,3. Sus principales eferencias directas son al...,4. Recibe aferencias exclusivamente del área ...,5. Su lesión no impide realizar movimientos v...,1,2012
5,6. Es un núcleo motor troncoencefálico:,1. Putamen.,2. Núcleo rojo.,3. Fastigial.,4. Caudado.,5. Dentado.,2,2012
6,7. En el reflejo tendinoso de Golgi se inhiben:,1. Los órganos tendinosos de Golgi.,2. Las fibras aferentes Ia.,3. Las fibras aferentes Ib.,4. Las motoneuronas α.,5. Las motoneuronas γ.,4,2012
7,8. Las fibras trepadoras de la corteza cerebel...,1. Son inhibidoras.,2. Proceden de los núcleos pontinos.,3. Se proyectan en las células granulosas.,4. Sinaptan directamente con las células de P...,5. Forman las fibras paralelas.,4,2012
8,"9. Los canales semicirculares, el utrículo y e...",1. Presentar otolitos.,2. Detectar la aceleración lineal.,3. Detectar la aceleración angular.,4. Tener un extremo llamado ampolla.,5. Estar llenos de endolinfa.,5,2012
9,10. Son amielínicas las fibras:,1. A alfa.,2. A beta.,3. A gamma.,4. B.,5. C.,5,2012


# Data Loading

## Saving in csv file

The `processed_exam_df` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [39]:
processed_exam_df.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [40]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [41]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [42]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df.itertuples():
        cur.execute(query_question, (str(question[8]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:7]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()